# Deploying Bokeh Apps

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')

## Purpose

HoloViews is an incredibly convenient way of working interactively and exploratively within a notebook or commandline context. However, once you have implemented a polished interactive dashboard or some other complex interactive visualization, you will often want to deploy it outside the notebook to share with others who may not be comfortable with the notebook interface. 

In the simplest case, to visualize some HoloViews container or element `obj`, you can export it to a standalone HTML file for sharing using the `save` method of the Bokeh renderer:

```
hv.renderer('bokeh').save(obj,'out')
```

This command will generate a file `out.html` that you can put on any web server, email directly to colleagues, etc.; it is fully self-contained and does not require any Python server to be installed or running.  

Unfortunately, a static approach like this cannot support any HoloViews object that uses DynamicMap (either directly or via operations that return DynamicMaps like `decimate`, `datashade`, and `rasterize`).  Anything with DynamicMap requires a live, running Python server to dynamically select and provide the data for the various parameters that can be selected by the user. Luckily, when you need a live Python process during the visualization, the [Bokeh server](http://bokeh.pydata.org/en/latest/docs/user_guide/server.html) provides a very convenient way of deploying HoloViews plots and interactive dashboards in a scalable and flexible manner. The Bokeh server allows all the usual interactions that HoloViews lets you define and more including:

* responding to plot events and tool interactions via [Linked Streams](./12-Custom_Interactivity.ipynb)
* generating and interacting with plots via the usual widgets that HoloViews supports for HoloMap and DynamicMap objects.
* using periodic and timeout events to drive plot updates
* combining HoloViews plots with custom Bokeh plots to quickly write highly customized apps.

## Overview

In this guide we will cover how we can deploy a Bokeh app from a HoloViews plot in a number of different ways:

1. Inline from within the Jupyter notebook

2. Starting a server interactively and open it in a new browser window.

3. From a standalone script file

4. Combining HoloViews and Bokeh models to create a more customized app

If you have read a bit about HoloViews you will know that HoloViews objects are not themselves plots, instead they contain sufficient data and metadata allowing them to be rendered automatically in a notebook context. In other words, when a HoloViews object is evaluated a backend specific ``Renderer`` converts the HoloViews object into Bokeh models, a Matplotlib figure or a Plotly graph. This intermediate representation is then rendered as an image or as HTML with associated Javascript, which is what ends up being displayed.

## The workflow

The most convenient way to work with HoloViews is to iteratively improve a visualization in the notebook. Once you have developed a visualization or dashboard that you would like to deploy you can use the ``BokehRenderer`` to export the visualization as illustrated above, or you can deploy it as a Bokeh server app. 

Here we will create a small interactive plot, using [Linked Streams](./12-Custom_Interactivity.ipynb), which mirrors the points selected using box- and lasso-select tools in a second plot and computes some statistics:

In [ ]:
%%opts Points [tools=['box_select', 'lasso_select']]

# Declare some points
points = hv.Points(np.random.randn(1000,2 ))

# Declare points as source of selection stream
selection = hv.streams.Selection1D(source=points)

# Write function that uses the selection indices to slice points and compute stats
def selected_info(index):
    arr = points.array()[index]
    if index:
        label = 'Mean x, y: %.3f, %.3f' % tuple(arr.mean(axis=0))
    else:
        label = 'No selection'
    return points.clone(arr, label=label).options(color='red')

# Combine points and DynamicMap
layout = points + hv.DynamicMap(selected_info, streams=[selection])
layout

<img src='https://assets.holoviews.org/gifs/examples/streams/bokeh/point_selection1d.gif'></img>

#### Working with the BokehRenderer

When working with Bokeh server or wanting to manipulate a backend specific plot object you will have to use a HoloViews ``Renderer`` directly to convert the HoloViews object into the backend specific representation. Therefore we will start by getting a hold of a ``BokehRenderer``:

In [ ]:
renderer = hv.renderer('bokeh')
print(renderer)

```python
BokehRenderer()
```

All ``Renderer`` classes in HoloViews are so called ParameterizedFunctions; they provide both classmethods and instance methods to render an object. You can easily create a new ``Renderer`` instance using the ``.instance`` method:

In [ ]:
renderer = renderer.instance(mode='server')

Renderers can also have different modes. In this case we will instantiate the renderer in ``'server'`` mode, which tells the Renderer to render the HoloViews object to a format that can easily be deployed as a server app. Before going into more detail about deploying server apps we will quickly remind ourselves how the renderer turns HoloViews objects into Bokeh models.

### Figures

The BokehRenderer converts the HoloViews object to a HoloViews ``Plot``, which holds the Bokeh models that will be rendered to screen. As a very simple example we can convert a HoloViews ``Image`` to a HoloViews plot:

In [ ]:
hvplot = renderer.get_plot(layout)
print(hvplot)

```
<LayoutPlot LayoutPlot01808>
```

Using the ``state`` attribute on the HoloViews plot we can access the Bokeh ``Column`` model, which we can then work with directly.

In [ ]:
hvplot.state

**Column**(	id = '5a8b7949-decd-4a96-b1f8-8f77ec90e5bf', …)


In the background this is how HoloViews converts any HoloViews object into Bokeh models, which can then be converted to embeddable or standalone HTML and be rendered in the browser. This conversion is usually done in the background using the ``figure_data`` method:

In [ ]:
html = renderer._figure_data(hvplot)

### Bokeh Documents

In Bokeh the [``Document``](http://bokeh.pydata.org/en/latest/docs/reference/document.html) is the basic unit at which Bokeh models (such as plots, layouts and widgets) are held and serialized. The serialized JSON representation is then sent to BokehJS on the client-side browser. When in ``'server'`` mode the BokehRenderer will automatically return a server Document:

In [ ]:
renderer(layout)

```
(<bokeh.document.Document at 0x11afc7590>,
 {'file-ext': 'html', 'mime_type': u'text/html'})
```

We can also easily use the ``server_doc`` method to get a Bokeh ``Document``, which does not require you to make an instance in 'server' mode.

In [ ]:
doc = renderer.server_doc(layout)
doc.title = 'HoloViews App'

## Deploying with ``bokeh serve``

Deployment from a script with `bokeh serve` is one of the most common ways to deploy a Bokeh app. Any ``.py`` or ``.ipynb`` file that attaches a plot to Bokeh's ``curdoc`` can be deployed using ``bokeh serve``. The easiest way to do this is using the ``BokehRenderer.server_doc`` method, which accepts any HoloViews object generates the appropriate Bokeh models and then attaches them to ``curdoc``. See below to see a full standalone script:

```python
import numpy as np
import holoviews as hv
import holoviews.plotting.bokeh

renderer = hv.renderer('bokeh')

points = hv.Points(np.random.randn(1000,2 )).options(tools=['box_select', 'lasso_select'])
selection = hv.streams.Selection1D(source=points)

def selected_info(index):
    arr = points.array()[index]
    if index:
        label = 'Mean x, y: %.3f, %.3f' % tuple(arr.mean(axis=0))
    else:
        label = 'No selection'
    return points.clone(arr, label=label).options(color='red')

layout = points + hv.DynamicMap(selected_info, streams=[selection])

doc = renderer.server_doc(layout)
doc.title = 'HoloViews App'
```

In just a few steps, i.e. by our plot to a Document ``renderer.server_doc`` we have gone from an interactive plot  which we can iteratively refine in the notebook to a deployable Bokeh app. Note also that we can also deploy an app directly from a notebook. By adding ``BokehRenderer.server_doc(holoviews_object)`` to the end of the notebook any regular ``.ipynb`` file can be made into a valid Bokeh app, which can be served with ``bokeh serve example.ipynb``.

In addition to starting a server from a script we can also start up a server interactively, so let's do a quick deep dive into Bokeh ``Application`` and ``Server`` objects and how we can work with them from within HoloViews.

## Bokeh Applications and Server

A Bokeh ``Application`` encapsulates a Document and allows it to be deployed on a Bokeh server. The ``BokehRenderer.app`` method provides an easy way to create an ``Application`` and either display it immediately in a notebook or manually include it in a server app.

To let us try this out we'll define a slightly simpler plot to deploy as a server app. We'll define a ``DynamicMap`` of a sine ``Curve`` varying by frequency, phase and an offset.

In [ ]:
def sine(frequency, phase, amplitude):
    xs = np.linspace(0, np.pi*4)
    return hv.Curve((xs, np.sin(frequency*xs+phase)*amplitude)).options(width=800)

ranges = dict(frequency=(1, 5), phase=(-np.pi, np.pi), amplitude=(-2, 2), y=(-2, 2))
dmap = hv.DynamicMap(sine, kdims=['frequency', 'phase', 'amplitude']).redim.range(**ranges)

app = renderer.app(dmap)
print(app)

```
<bokeh.application.application.Application object at 0x11c0ab090>
```

Once we have a Bokeh Application we can manually create a ``Server`` instance to deploy it. To start a ``Server`` instance we simply define a mapping between the URL paths and apps that we want to deploy. Additionally we define a port (defining ``port=0`` will use any open port).

In [ ]:
from bokeh.server.server import Server

server = Server({'/': app}, port=0)

Next we can define a callback on the IOLoop that will open the server app in a new browser window and actually start the app (and if outside the notebook the IOLoop):

In [ ]:
server.start()
server.show('/')

# Outside the notebook ioloop needs to be started
# from tornado.ioloop import IOLoop
# loop = IOLoop.current()
# loop.start() 

After running the cell above you should have noticed a new browser window popping up displaying our plot. Once you are done playing with it you can stop it with:

In [ ]:
server.stop()

The ``BokehRenderer.app`` method allows us to the same thing automatically (but less flexibly) using the ``show=True`` and ``new_window=True`` arguments:

In [ ]:
server = renderer.app(dmap, show=True, new_window=True)

<img width='80%' src="https://assets.holoviews.org/gifs/guides/user_guide/Deploying_Bokeh_Apps/bokeh_server_new_window.png"></img>

We will once again stop this Server before continuing:

In [ ]:
server.stop()

## Inlining apps in the notebook

Instead of displaying our app in a new browser window and manually creating a ``Server`` instance we can also display an app inline in the notebook simply by supplying the ``show=True`` argument to the ``BokehRenderer.app`` method. The server app will be killed whenever you rerun or delete the cell that contains the output. Additionally, if your Jupyter Notebook server is not running on the default address or port (``localhost:8888``) supply the websocket origin, which should match the first part of the URL of your notebook:

In [ ]:
renderer.app(dmap, show=True, websocket_origin='localhost:8888')

<img width='80%' src='https://assets.holoviews.org/gifs/guides/user_guide/Deploying_Bokeh_Apps/bokeh_server_inline_simple.gif'></img>

## Periodic callbacks

One of the most important features of deploying apps is the ability to attach asynchronous, periodic callbacks, which update the plot. The simplest way of achieving this is to attach a ``Counter`` stream on the plot which is incremented on each callback. As a simple demo we'll simply compute a phase offset from the counter value, animating the sine wave:

In [ ]:
def sine(counter):
    phase = counter*0.1%np.pi*2
    xs = np.linspace(0, np.pi*4)
    return hv.Curve((xs, np.sin(xs+phase))).options(width=800)

dmap = hv.DynamicMap(sine, streams=[hv.streams.Counter()])

app = renderer.app(dmap, show=True, websocket_origin='localhost:8888')

<img width='80%' src='https://assets.holoviews.org/gifs/guides/user_guide/Deploying_Bokeh_Apps/bokeh_server_periodic.gif'></img>

Once we have created the app we can start a periodic callback with the ``periodic`` method on the ``DynamicMap``. The first argument to the method is the period and the second argument the number of executions to trigger (we can set this value to ``None`` to set up an indefinite callback). As soon as we start this callback you should see the Curve above become animated.

In [ ]:
dmap.periodic(0.1, 100)

## Combining HoloViews and Bokeh Plots/Widgets

While HoloViews provides very convenient ways of creating an app it is not as fully featured as Bokeh itself is. Therefore we often want to extend a HoloViews based app with Bokeh plots and widgets created directly using the Bokeh API. Using the ``BokehRenderer`` we can easily convert a HoloViews object into a Bokeh model, which we can combine with other Bokeh models as desired.

To see what this looks like we will use the sine example again but this time connect a [Stream](./11-Responding_to_Events.ipynb) to a manually created Bokeh slider widget and play button. To display this in the notebook we will reuse what we learned about creating a ``Server`` instance using a ``FunctionHandler``, you can of course run this in a script by calling the ``modify_doc`` function with with the ``Document`` returned by the Bokeh ``curdoc()`` function.

In [ ]:
import numpy as np
import holoviews as hv

from bokeh.io import show
from bokeh.layouts import layout
from bokeh.models import Slider, Button

renderer = hv.renderer('bokeh').instance(mode='server')

# Create the holoviews app again
def sine(phase):
    xs = np.linspace(0, np.pi*4)
    return hv.Curve((xs, np.sin(xs+phase))).options(width=800)

stream = hv.streams.Stream.define('Phase', phase=0.)()
dmap = hv.DynamicMap(sine, streams=[stream])

# Define valid function for FunctionHandler
# when deploying as script, simply attach to curdoc
def modify_doc(doc):
    # Create HoloViews plot and attach the document
    hvplot = renderer.get_plot(dmap, doc)

    # Create a slider and play buttons
    def animate_update():
        year = slider.value + 0.2
        if year > end:
            year = start
        slider.value = year

    def slider_update(attrname, old, new):
        # Notify the HoloViews stream of the slider update 
        stream.event(phase=new)
        
    start, end = 0, np.pi*2
    slider = Slider(start=start, end=end, value=start, step=0.2, title="Phase")
    slider.on_change('value', slider_update)
    
    callback_id = None

    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = doc.add_periodic_callback(animate_update, 50)
        else:
            button.label = '► Play'
            doc.remove_periodic_callback(callback_id)
    button = Button(label='► Play', width=60)
    button.on_click(animate)
    
    # Combine the holoviews plot and widgets in a layout
    plot = layout([
    [hvplot.state],
    [slider, button]], sizing_mode='fixed')
    
    doc.add_root(plot)
    return doc

# To display in the notebook
show(modify_doc, notebook_url='localhost:8888')

# To display in a script
#    doc = modify_doc(curdoc()) 

<img width='80%' src='https://assets.holoviews.org/gifs/guides/user_guide/Deploying_Bokeh_Apps/bokeh_server_play.gif'></img>

If you had trouble following the last example, you will have noticed how verbose things can get when we drop down to the Bokeh API. The ability to customize the plot comes at the cost of additional complexity. However when we need it, the additional flexibility of composing plots manually is there.